In [1]:
import os
os.chdir('/Users/omii/Desktop/Brain-Tumor-Classification/')

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub 
import os
import mlflow.keras
import mlflow


# os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = './src/keys/tensile-topic-424308-d9-7418db5a1c90'

keyfile_path = 'src/keys/tensile-topic-424308-d9-7418db5a1c90.json'  #change as per your keyfile path

# Checking if file exists
if not os.path.exists(keyfile_path):
    raise FileNotFoundError(f"The file '{keyfile_path}' does not exist. Please check the path.")

# Set the environment variable to point to the service account key file
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = keyfile_path
os.environ['MLFLOW_GCS_BUCKET'] = 'ml-flow-remote-tracker-bucket'

mlflow.set_tracking_uri("http://35.231.231.140:5000/")
mlflow.set_experiment("Brain-Tumor-Classification")



physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print("TensorFlow with Metal enabled with {} GPUs".format(len(physical_devices)))
    except RuntimeError as e:
        print(e)


def preprocessing_for_training():
    path = './data/Training/'
    train_datagen = ImageDataGenerator(
        rescale=1.0/255,
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest'
    )
    train_generator = train_datagen.flow_from_directory(
        path,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle=True,
        seed=42
    )
    return train_generator

def preprocessing_for_testing_inference(batchSize):
    path = './data/Testing/'
    test_val_datagen = ImageDataGenerator(rescale=1.0/255)
    test_generator = test_val_datagen.flow_from_directory(
        path,
        target_size=(224, 224),
        batch_size=batchSize,
        class_mode='categorical',
        shuffle=False
    )
    return test_generator


model = models.Sequential([
    layers.InputLayer(shape=(224, 224, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(4, activation='sigmoid')
])
model.summary()

train_dataset = preprocessing_for_training()
validation_dataset = preprocessing_for_testing_inference(batchSize=32)


with mlflow.start_run() as run:
    # Log parameters
    mlflow.log_param("batch_size", 32)
    mlflow.log_param("epochs", 10)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_function", "binary_crossentropy")
    mlflow.log_param("metrics", "accuracy")

    model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

    history = model.fit(
        train_dataset,
        validation_data=validation_dataset,
        epochs=10
    )

    mlflow.keras.log_model(model, "model")

# Log metrics
for epoch in range(10):
    mlflow.log_metrics({
        "train_loss": history.history['loss'][epoch],
        "train_accuracy": history.history['accuracy'][epoch],
        "val_loss": history.history['val_loss'][epoch],
        "val_accuracy": history.history['val_accuracy'][epoch]
    }, step=epoch)


mlflow.log_artifacts('./data/Training/', artifact_path="training_data")
mlflow.log_artifacts('./data/Testing/', artifact_path="testing_data")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,476 (42.61 MB)

 Trainable params: 11,169,476 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
Epoch 1/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 520s 3s/step - accuracy: 0.5043 - loss: 0.4717 - val_accuracy: 0.7635 - val_loss: 0.3178
Epoch 2/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 294s 2s/step - accuracy: 0.7701 - loss: 0.2702 - val_accuracy: 0.6522 - val_loss: 0.4666
Epoch 3/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.7971 - loss: 0.2442 - val_accuracy: 0.7689 - val_loss: 0.2786
Epoch 4/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.8333 - loss: 0.2117 - val_accuracy: 0.7559 - val_loss: 0.3203
Epoch 5/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 278s 2s/step - accuracy: 0.8221 - loss: 0.2127 - val_accuracy: 0.8154 - val_loss: 0.2209
Epoch 6/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.8662 - loss: 0.1723 - val_accuracy: 0.8253 - val_loss: 0.2023
Epoch 7/10
179/179 ━━━━━━━━━━━━━━━━━━━━ 277s 2s/step - accuracy: 0.8570 - loss: 0.1788 - val_accuracy: 0.8101 - val_loss: 0.2303
Epoch 8/10
17

2024/06/18 13:57:39 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
